In [ ]:
library(tidyverse)
library(data.table)
library(dplyr)
library(stringr)
library(readr)
library(tidyverse)
library(RcppParallel)
library(quanteda)
library(janeaustenr)
library(dplyr)
library(tidyr)
library(tidytext)

In [ ]:
twitter_csv <- fread("UBC-ISD.csv", encoding="UTF-8")

In [ ]:
terms <- c('fuck', 'dirty', 'stupid', 'idiot', 'asshole', 'bastard', 'nasty', 'whore', 'slut', 'skank', 'cunt', 'bitch', 'motherfucker', 'go to hell', 'wimp', 'coward', 'moron', 'traitor', 'shut up', 'shut the fuck up', 'stfu', 'piece of shit', 'ignorant', 'clown', 'ccpvirus', 'chinese coronavirus', 'chinavirus', 'china virus', 'fuckchina', 'nukechina', 'bombchina', 'deathtochina', 'chinesevirus', 'fake lab wuhan', 'china weapon', 'china bioweapon', 'zionazi', 'Soros virus', 'blood libel', 'nwovirus', 'new world order', 'sacrifice children', 'jew world order', 'liar', 'yellow star', 'israel virus', 'israelvirus', 'rothschild covid', 'kalergi plan', 'jail', 'locked up', 'evil.', 'is evil', 'so evil', 'pure evil', 'an evil', 'evil wom', 'evil man', 'evil witch', 'is evil', 'are evil', 'that evil', 'very evil', 'tyrant', 'tyranny', 'dictator', 'tv doctor', 'twitter doctor', 'murder', 'murderer', 'eugenic', 'satan', 'Nazi', 'non-human', 'sterile', 'slave', 'crooked', 'corrupt', 'shill', 'liar', 'lying', 'killer', 'fraud', 'big pharma', 'pharma puppet', 'Nuremberg', 'crimes against humanity', 'genocide', 'child abuse', 'covid hoax', 'covid fake', 'virus hoax', 'gates virus', 'bioweapon', 'secret elite', 'depopulation', 'save our children', 'scamdemic', 'plandemic')
terms <- paste0(" ", terms)

In [ ]:
terms

In [ ]:

twitter_data <- twitter_csv |>
select(starts_with("twitter"))

colnames(twitter_data) <- str_replace(colnames(twitter_data), "twitter.tweet/", "")
colnames(twitter_data) <- str_replace(colnames(twitter_data), "twitter.user/", "")


colnames(twitter_data)

In [ ]:
twitter_info<- twitter_data |>
select("text", "created")

In [ ]:
pattern <- paste(terms, collapse = "|")

twitter_info <- twitter_info |>
  mutate(negative = as.integer(grepl(pattern, text, ignore.case = TRUE)))

In [ ]:
twitter_info

In [ ]:
extract_hashtags <- function(x) {
  hashtags <- str_extract_all(x, "#\\S+")
  return(unlist(hashtags))
}

In [ ]:
library(tidyverse)

In [ ]:
twitter_info <- twitter_info |>
  mutate(hashtags = lapply(text, extract_hashtags))|>
  unnest(hashtags)|>
  mutate(hashtags = str_replace_all(hashtags, "\\.$", ""))|>
  filter(hashtags != "character(0)")

In [ ]:
twitter_graphing <- twitter_info |>
select(created, negative)|>
mutate(created = round_date(created, unit = "day"))|>
group_by(created)|>
summarise(freq = n())|>
ungroup()

In [ ]:
twitter_graphing

In [ ]:
options(repr.plot.width=12, repr.plot.height=6) 

graph <- twitter_graphing|>
    ggplot(aes(x=created, y=freq))+geom_point(alpha=0.3, color="#f08f84")+
    geom_smooth(aes(), span=0.1, method="loess", se=FALSE, size=1, color="#bd3526")+
    scale_x_datetime(date_breaks = "1 month", date_labels = "%b %Y")+
    theme(aspect.ratio=1/2, axis.text.x = element_text(angle=45, hjust = 1))+
    theme_bw()+
    labs(
        title="Frequency of toxic tweets in ISD Dataset", 
       subtitle="Dictionary-Based Approach",
       y="Frequency (# of Tweets)", 
       x="Date",
       color=NULL
       )
graph

ggsave("toxic_tweets_dictionary_span.png", width=10, height=5)